# MDPs, Trajetórias e Retornos


In [1]:
import gym
import time
import numpy as np

# vamos focar nesses três ambientes por serem mais simples
env = gym.make("MountainCar-v0")
#env = gym.make("CartPole-v1")
#env = gym.make("Taxi-v3")

c:\Users\Pablo\anaconda3\envs\rlx\lib\site-packages\gym\utils\passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


![Figura mostrando interação agente(política)-ambiente](figura_mdp.png "Interação agente-ambiente")

# 1. Episódio e Trajetória

Um *episódio* é uma execução da tarefa (ou do ambiente gym). 

E a *trajetória* é a sequência de estados (observações), ações e recompensas do episódio. Assumindo um episódio de $T$ passos (ações aplicadas):

$S_0 \rightarrow A_0 \rightarrow R_1 \rightarrow S_1 \rightarrow A_1 \rightarrow R_2 \rightarrow S_2 \rightarrow \cdots S_{T-1} \rightarrow A_{T-1} \rightarrow R_T \rightarrow S_T$

Vamos ilustrar um episódio em um MDP usando o ambiente *"env"* escolhido no código acima. 

Estamos assumindo que o episódio encerrou de fato (chegou em um estado final) em *$n$="TOTAL_STEPS"* passos.

In [2]:
TOTAL_STEPS = 5

i = 0
obs = env.reset()
print(f"S0 = {obs}")

done = False

# roda apenas alguns passos
for i in range(0,TOTAL_STEPS):
    #env.render()
    action = env.action_space.sample()
    print(f" A{i} = {action}")

    next_obs, reward, done, info = env.step(action)

    print(f"  R{i+1} = {reward}")
    print(f"S{i+1} = {next_obs}")

    obs = next_obs
    time.sleep(0.1)

env.close()

S0 = [-0.4965765  0.       ]
 A0 = 1
  R1 = -1.0
S1 = [-4.9677894e-01 -2.0244533e-04]
 A1 = 0
  R2 = -1.0
S2 = [-0.4981823  -0.00140338]
 A2 = 0
  R3 = -1.0
S3 = [-0.5007761  -0.00259382]
 A3 = 1
  R4 = -1.0
S4 = [-0.503541   -0.00276485]
 A4 = 0
  R5 = -1.0
S5 = [-0.5074562 -0.0039152]


Os detalhes do *episódio* que mostramos acima são chamamos de *trajectory* ou *rollout*.

Dependendo do algoritmo, vamos precisar analisar essas informações em trios (S,A,R) ou quádruplas (S,A,R,S) ou até quíntuplas (S,A,R,S',A').

Abaixo, vamos agrupar e guardar em trio, para preparar para o algoritmo Monte Carlo.

In [3]:
TOTAL_STEPS = 5

i = 0
obs = env.reset()
trajectory = [] 

done = False

for i in range(0,TOTAL_STEPS):
    #env.render()
    action = env.action_space.sample()

    next_obs, reward, done, info = env.step(action)

    trajectory.append( (obs, action, reward) )

    obs = next_obs
    time.sleep(0.1)

env.close()
#trajectory.append( (obs, None, None) )

print("Trajetória como sequência de trios (STATE, ACTION, REWARD):")
print(trajectory)

Trajetória como sequência de trios (STATE, ACTION, REWARD):
[(array([-0.5791818,  0.       ], dtype=float32), 0, -1.0), (array([-0.57976687, -0.00058506], dtype=float32), 2, -1.0), (array([-0.57893264,  0.00083421], dtype=float32), 2, -1.0), (array([-0.5766853 ,  0.00224731], dtype=float32), 1, -1.0), (array([-0.57404155,  0.00264378], dtype=float32), 1, -1.0)]


# 2.Calcular os Retornos

O *retorno (final)* $G$ é uma medida da recompensa total obtida ao longo de um episódio. 

Em um MDP, o objetivo é otimizar o valor médio de $G$, para infinitos episódios.

Para isso, vamos assumir a política abaixo, que escolhe a ação $0$ com 50% de probabilidade, ou outra ação, caso contrário.


In [4]:
num_actions = env.action_space.n

def policy(state):
    x = np.random.random()
    if x <= 0.5:
        return 0
    else:
        return np.random.randint(1, num_actions)


### 2.1 Retorno final do episódio ($G$)



Para um episódio com $n$ passos, o *retorno* (não-descontado) é calculado assim:

$ G = R_1 + R_2 + R_3 + \cdots + R_n = \displaystyle\sum_{i=1}^{n} R_i$

In [5]:
i = 0
obs = env.reset()
trajectory = [] 

done = False
sum_rewards = 0.0

while not done:
    #env.render()
    action = policy(obs)

    next_obs, reward, done, info = env.step(action)
    sum_rewards += reward

    obs = next_obs
    time.sleep(0.1)

env.close()
#trajectory.append( (obs, None, None) )

print("Retorno não-descontado:", sum_rewards)

Retorno não-descontado: -200.0


O mais usado é o *retorno descontado* de um episódio.

Neste caso, $G$ é uma soma que "atenua" recompensas mais distantes, valorizando mais as recompensas iniciais. (Você prefere receber 100 reais agora, de uma vez, ou em 100 parcelas de 1 real?)

Para isso, a cada passo, a recompensa tem uma redução dada por um parâmetro $\gamma\;$, tal que $0 < \gamma \leq 1$.

Para um episódio com $n$ passos, o *retorno descontado* é calculado assim:

$ G = R_1 + \gamma R_2 + \gamma^2 R_3 + \cdots + \gamma^{(n-1)} R_n = \displaystyle\sum_{t=1}^{n} \gamma^{(t-1)} R_t$

In [6]:
i = 0
obs = env.reset()
trajectory = [] 

done = False
discounted_sum_rewards = 0.0
gamma = 0.95

step = 0

while not done:
    #env.render()
    action = policy(obs)

    next_obs, reward, done, info = env.step(action)
    discounted_sum_rewards += (gamma ** step) * reward
    step += 1
    
    obs = next_obs
    time.sleep(0.1)

env.close()
#trajectory.append( (obs, None, None) )

print("Retorno descontado:", discounted_sum_rewards)

Retorno descontado: -19.999298946675


### 2.2 Retornos intermediários a cada passo ($G_i$)



Também podemos calcular um retorno parcial, de um certo episódio, a partir de um passo específico $i$ do episódio:

- $ G_0 = R_1 + \gamma R_2 + \gamma^2 R_3 + \gamma^3 R_4 + \gamma^4 R_5 + \cdots \;\;= G$ 
- $ G_1 = \;\;\;\;\;\;\;\;\;  R_2 + \;\gamma R_3 + \;\gamma^2 R_4 + \;\gamma^3 R_5  + \cdots $ 
- $ G_2 = \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; R_3 + \;\gamma R_4 + \;\gamma^2 R_5 + \cdots $ 
- $\cdots$ 
- $ G_n = 0 $

Propriedade:
- ??

In [7]:
#$ G_{i-1} = r_i + \gamma G_i $

# 3. Funções de Valor

São funções que não fazem parte da essência de um MDP, mas são úteis para criar algoritmos.

Veremos dois tipos de função de valor. Ambas são calculadas a partir dos retornos intermediários.

### 3.1 Função de valor do estado V(s)

Calcula o retorno esperado a partir de todo estado $s$.

In [8]:
# fica como exercício

### 3.2 Função de valor do estado-ação Q(s,a)

Calcula o retorno esperado a partir do par $(s,a)$.

*"Quando estava no estado $s$ e fez a ação $a$, qual a recompensa futura esperada?"*

In [9]:
# vamos fazer?

# 4. Introdução aos Métodos Baseados em Q-table

Suponha que você partiu de uma política qualquer (provavelmente ruim) e calculou o *Q* dessa política.

De que forma poderíamos melhorá-la?